<a href="https://colab.research.google.com/github/IVReddy81/pyspark/blob/main/RDD%20Transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark=SparkSession.builder.master("local").appName("king").getOrCreate()
sc=spark.sparkContext
#map
a=sc.parallelize([1,2,3,4])
rmap=a.map(lambda x:(x,1))
rmap.collect()

[(1, 1), (2, 1), (3, 1), (4, 1)]

In [ ]:
#flatMap
rflatMap=a.flatMap(lambda x:(x,1))
rflatMap.collect()

[1, 1, 2, 1, 3, 1, 4, 1]

In [ ]:
#filter
b=a.filter(lambda x: x%2==0).collect()
print(b)

[2, 4]


In [ ]:
#sample -> takes particular fraction of output from input
#sample rdd.sample([with Replacement]True/False,Percentage of data(0.1 to 1.0),seed(an integer))
#true= duplicate values printed
#False= no duplicates
#give same seed two times to get same output
a=sc.parallelize(range(100))
a.sample(False,0.1,100).collect()

[27, 35, 47, 59, 62, 80, 88, 90, 96, 98]

In [ ]:
#union -> append one RDD to another RDD ex: b append to a
b=sc.parallelize([1,2,5,7,4,0]) #a=[1, 2, 3, 4] b=[1, 2, 5, 7, 4, 0]
a.union(b).collect()  #a union b

[1, 2, 3, 4, 1, 2, 5, 7, 4, 0]

In [ ]:
#union -> append one RDD to another RDD ex: a append to b
b.union(a).collect() #b union a // a=[1, 2, 3, 4] b=[1, 2, 5, 7, 4, 0]

[1, 2, 5, 7, 4, 0, 1, 2, 3, 4]

In [ ]:
##Intersection -> takes common values from both RDD's
a.intersection(b).collect()  #a=[1, 2, 3, 4] b=[1, 2, 5, 7, 4, 0]

[2, 4, 1]

In [ ]:
#Intersection -> takes common values from both RDD's
b.intersection(a).collect()  #a=[1, 2, 3, 4] b=[1, 2, 5, 7, 4, 0]

[2, 4, 1]

In [ ]:
#distinct -> drops all duplicates in list/rdd
c=sc.parallelize([1,2,1,3,4,2,4,5,7,2,1])
c.distinct().collect()

[1, 2, 3, 4, 5, 7]

In [ ]:
#sortBy asc
a=sc.parallelize([1,2,1,3,4,2,4,5,7,2,1])
a.sortBy(lambda x: x,True).collect()  #True= ascending, #False= descending

[1, 1, 1, 2, 2, 2, 3, 4, 4, 5, 7]

In [ ]:
#sortBy  desc
b=sc.parallelize([1,2,1,3,4,2,4,5,7,2,1])
b.sortBy(lambda x: x,False).collect()  

[7, 5, 4, 4, 3, 2, 2, 2, 1, 1, 1]

In [ ]:
#mapPartitions --> sum all the elements and provides it as
a=sc.parallelize([1,2,3,4,5,2,1,3,2,9],3)  #it will sum all the elements in list to 3 elements
def f(iterator):       #1+2+3=6, 4+5+2=11, 1+3+2+9=15 
  yield sum(iterator)
a.mapPartitions(f).collect()

[6, 11, 15]

In [ ]:
#mapPartitionsWithIndex
a=sc.parallelize([1,2,3,4,5,2,1,3,2,9],6)  
def f(splitindex,iterator):        
  yield splitindex
a.mapPartitionsWithIndex(f).sum()

15

In [ ]:
#groupBy
a=sc.parallelize([1,2,3,4,5,2,1,3,2,9])
y=a.groupBy(lambda x:x%2).collect()
print(sorted([(x,sorted(y)) for x,y in y])) 

[(0, [2, 2, 2, 4]), (1, [1, 1, 3, 3, 5, 9])]


In [ ]:
#zip -> combines (a[index],b[index])
a=sc.parallelize([1,2,3,4,5])
b=sc.parallelize([3,2,4,5,6])
a.zip(b).collect()

[(1, 3), (2, 2), (3, 4), (4, 5), (5, 6)]

In [ ]:
#zipWithIndex
a=sc.parallelize([1,2,3,4,5],6) #1/2/3/4/5/6...... it will give same ouptput
a.zipWithIndex().collect()

[(1, 0), (2, 1), (3, 2), (4, 3), (5, 4)]

In [ ]:
#repartition -> to inc or dec no.of partitions(full shufffle across all partitions)
a=sc.parallelize(range(20))
a.repartition(3).glom().collect()

[[], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

In [ ]:
sc.parallelize(range(12),5).glom().collect()  #repartition (both are same)

[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19]]

In [ ]:
#coalesce -> to dec num of partitions(combines existing partitions to avoid full shuffle)
a=sc.parallelize(range(12),5)   #sc.parallelize(range(12),5).coalesce(2).glom().collect()
a.coalesce(2).glom().collect()  #reduces 5 partitions(above) to two partitions

[[0, 1, 2, 3], [4, 5, 6, 7, 8, 9, 10, 11]]